In [58]:
1+1

2

In [59]:
from typing import Annotated
from langchain_ollama import ChatOllama
from langchain_core.messages import AnyMessage,AIMessage

from langgraph.graph import StateGraph,END,START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt,Command
from dotenv import load_dotenv
from langgraph.types import interrupt,Command
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage

In [60]:
llm=ChatOllama(model='llama3.2:1b',temperature=0.2,num_predict=256)

In [61]:
class ChatState(TypedDict):
    message:Annotated[list[BaseMessage],add_messages]

In [62]:
def chat_node(state:ChatState):
    decision=interrupt({
        "type":"approval",
        "reason":"Model is abouth to answer the user question.",
        "question":state["message"][-1].content,
        "instruction":"Approved the question ?yes/no"
        })
    
    if decision['approved']=='no':
        return {"message":[AIMessage(content="NOt approved")]}
    else:
        responce=llm.invoke(state["message"])
        return {"message":[responce]}

In [63]:
graph=StateGraph(ChatState)

graph.add_node("chat",chat_node)

graph.add_edge(START,"chat")
graph.add_edge("chat",END)

checkpointer=MemorySaver()

# compile the flow

app=graph.compile(checkpointer=checkpointer)

In [64]:
config={"configurable":{"thread_id":"1234"}}


initial_input={
    "message":["user","Explain the gradient descent in simple term?"]
}

result=app.invoke(initial_input,config=config)

In [65]:
result

{'message': [HumanMessage(content='user', additional_kwargs={}, response_metadata={}, id='1c28b2f0-a18a-4f74-9634-29e6d9cc607a'),
  HumanMessage(content='Explain the gradient descent in simple term?', additional_kwargs={}, response_metadata={}, id='28e5c8c5-b0b9-4443-a407-50e336e00611')],
 '__interrupt__': [Interrupt(value={'type': 'approval', 'reason': 'Model is abouth to answer the user question.', 'question': 'Explain the gradient descent in simple term?', 'instruction': 'Approved the question ?yes/no'}, id='40db6d2fb3abe31aa42537b80e2ecc32')]}

In [66]:
message=result['__interrupt__'][0].value
message

{'type': 'approval',
 'reason': 'Model is abouth to answer the user question.',
 'question': 'Explain the gradient descent in simple term?',
 'instruction': 'Approved the question ?yes/no'}

In [67]:
user_input=input(f"\n Backend message  - {message} \n approved this question ? yes/no")

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019be14d-6210-7662-87d6-b45de2221414,id=019be14d-6210-7662-87d6-b45de2221414; trace=019be14d-6210-7662-87d6-b45de2221414,id=019be14d-6220-7cf1-8483-9288477c528a; trace=019be14d-6210-7662-87d6-b45de2221414,id=019be14d-6220-7cf1-8483-9288477c528a; trace=019be14d-6210-7662-87d6-b45de2221414,id=019be14d-6210-7662-87d6-b45de2221414


In [68]:
final_result=app.invoke(Command(resume={"approved":user_input}),config=config)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019be14d-6711-7980-b8ed-65c8ad0e8394,id=019be14d-6711-7980-b8ed-65c8ad0e8394; trace=019be14d-6711-7980-b8ed-65c8ad0e8394,id=019be14d-6716-7db2-80b3-071aa424fbd8; trace=019be14d-6711-7980-b8ed-65c8ad0e8394,id=019be14d-6717-7c41-8714-4ab324656075


In [70]:
final_result

{'message': [HumanMessage(content='user', additional_kwargs={}, response_metadata={}, id='1c28b2f0-a18a-4f74-9634-29e6d9cc607a'),
  HumanMessage(content='Explain the gradient descent in simple term?', additional_kwargs={}, response_metadata={}, id='28e5c8c5-b0b9-4443-a407-50e336e00611'),
  AIMessage(content='Gradient Descent is a simple concept to understand. Here\'s a simplified explanation:\n\n**Imagine you\'re trying to find the shortest path between two points on a map**\n\nIn this scenario, your goal is to minimize the distance (or "cost") between two specific locations.\n\n**Here\'s how Gradient Descent works:**\n\n1. **Start with an initial guess**: You choose a starting point for your path.\n2. **Calculate the direction of the shortest path**: Using the current location and the next location, you calculate the direction that would lead to the minimum distance (or cost).\n3. **Move in the calculated direction**: Move your path by the calculated amount in the direction of the sho